In [1]:
import re

# Use your input as provided.
input_references = """[1]	R. L. Siegel, K. D. Miller, N. S. Wagle, and A. Jemal, “Cancer statistics, 2023,” CA. Cancer J. Clin., vol. 73, no. 1, pp. 17–48, Jan. 2023, doi: 10.3322/caac.21763.
[2]	K. V. Sriram and R. H. Havaldar, “Analytical review and study on object detection techniques in the image,” Int. J. Model. Simul. Sci. Comput., vol. 12, no. 05, p. 2150031, Oct. 2021, doi: 10.1142/S1793962321500318.
[3]	L. Fan, H. Zhao, H. Zhao, H. Hu, and Z. Wang, “Survey of target detection based on deep convolutional neural networks,” Opt. Precis. Eng., vol. 28, no. 5, pp. 1152–1164, 2020, doi: 10.3788/ope.20202805.1152.
[4]	P. Viola and M. Jones, “Robust real-time face detection,” in Proceedings Eighth IEEE International Conference on Computer Vision. ICCV 2001, Vancouver, BC, Canada: IEEE Comput. Soc, 2001, pp. 747–747, doi: 10.1109/ICCV.2001.937709.
[5]	N. Dalal and B. Triggs, “Histograms of Oriented Gradients for Human Detection,” in 2005 IEEE Computer Society Conference on Computer Vision and Pattern Recognition (CVPR’05), San Diego, CA, USA: IEEE, 2005, pp. 886–893, doi: 10.1109/CVPR.2005.177.
[6]	M. Hussain, “YOLO-v1 to YOLO-v8, the Rise of YOLO and Its Complementary Nature toward Digital Manufacturing and Industrial Defect Detection,” Machines, vol. 11, no. 7, p. 677, Jun. 2023, doi: 10.3390/machines11070677.
[7]	G. Sharma, R. Dave, J. Sanadya, P. Sharma, and K. K. Sharma, “Various types and management of breast cancer: An overview,” J. Adv. Pharm. Technol. Res., vol. 1, no. 2, p. 109, 2010, doi: 10.4103/2231-4040.72251.
[8]	C. Li et al., “YOLOv6: A Single-Stage Object Detection Framework for Industrial Applications,” Sep. 07, 2022, arXiv: arXiv:2209.02976, doi: 10.48550/arXiv.2209.02976.
[9]	S. Zahia, D. Sierra-Sosa, B. Garcia-Zapirain, and A. Elmaghraby, “Tissue classification and segmentation of pressure injuries using convolutional neural networks,” Comput. Methods Programs Biomed., vol. 159, pp. 51–58, Jun. 2018, doi: 10.1016/j.cmpb.2018.02.018.
[10]	X. Sun, X. Wang, J. Liu, and H. Huang, “Classic YOLO Series Target Detection Algorithms and Their Application in Breast Cancer Detection,” J. Comput. Syst. Appl., vol. 32, no. 12, pp. 52–62, 2023, doi: 10.15888/j.cnki.csa.009351.
[11]	F. Prinzi, M. Insalaco, A. Orlando, S. Gaglio, and S. Vitabile, “A Yolo-Based Model for Breast Cancer Detection in Mammograms,” Cogn. Comput., vol. 16, no. 1, pp. 107–120, Jan. 2024, doi: 10.1007/s12559-023-10189-6.
[12]	P. K. Samanta, A. Basuli, N. K. Rout, and G. Panda, “Improved Breast Cancer Detection from Ultrasound Images Using YOLOv8 Model,” in 2023 IEEE 3rd International Conference on Applied Electromagnetics, Signal Processing, & Communication (AESPC), Bhubaneswar, India: IEEE, Nov. 2023, pp. 1–6, doi: 10.1109/AESPC59761.2023.10390341.
[13]	H. Gui et al., “FS-YOLOv9: A Frequency and Spatial Feature-Based YOLOv9 for Real-time Breast Cancer Detection,” Acad. Radiol., Oct. 2024, doi: 10.1016/j.acra.2024.09.048.
[14]	L. Zheng et al., “Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena,” Dec. 24, 2023, arXiv: arXiv:2306.05685, doi: 10.48550/arXiv.2306.05685.
[15]	A. Y. Yuan et al., “Hybrid deep learning network for vascular segmentation in photoacoustic imaging,” Biomed. Opt. Express, vol. 11, no. 11, p. 6445, Nov. 2020, doi: 10.1364/BOE.409246.
[16]	W. Al-Dhabyani, M. Gomaa, H. Khaled, and A. Fahmy, “Dataset of breast ultrasound images,” Data Brief, vol. 28, p. 104863, Feb. 2020, doi: 10.1016/j.dib.2019.104863.
"""

# Split input into individual references.
references_list = [ref.strip() for ref in input_references.strip().split("\n") if ref.strip()]

# --- Global Dictionary for Full Journal Names ---
def normalize_journal_key(name):
    return name.lower().replace(".", "").replace(" ", "")

# This dictionary maps a normalized version of a full journal name to its full form.
journal_full_names = {
    normalize_journal_key("CA: A Cancer Journal for Clinicians"): "CA: A Cancer Journal for Clinicians",
    normalize_journal_key("International Journal of Model, Simulation, and Scientific Computing"):
        "International Journal of Model, Simulation, and Scientific Computing",
    normalize_journal_key("Optics and Precision Engineering"): "Optics and Precision Engineering",
    normalize_journal_key("Journal of Advanced Pharmaceutical Technology & Research"):
        "Journal of Advanced Pharmaceutical Technology & Research",
    normalize_journal_key("Journal of Computer Systems Applications"): "Journal of Computer Systems Applications",
    normalize_journal_key("Cognitive Computation"): "Cognitive Computation",
    normalize_journal_key("Data Brief"): "Data Brief"
}

def check_full_journal_name(abbrev):
    """
    Check if the extracted journal name (abbrev) is the full name.
    Returns a tuple: (full_name, issue_note).
    If the journal is found in the dictionary and the extracted name does not equal the full name,
    an issue note is added.
    """
    normalized = abbrev.lower().replace(".", "").replace(" ", "")
    if normalized in journal_full_names:
        full = journal_full_names[normalized]
        if full != abbrev:
            return full, f"(Issues: Journal name should be full: {full})"
        else:
            return abbrev, ""
    else:
        return abbrev, "(Issues: Journal name not found in dictionary)"

# --- Citation Classification ---
def classify_citation(reference):
    ref_lower = reference.lower()
    if "arxiv" in ref_lower:
        return "arXiv"
    if re.search(r"thesis|dissertation", reference, re.IGNORECASE):
        return "Thesis/Dissertation"
    if "patent" in ref_lower:
        return "Patent"
    if re.search(r"private communication|unpublished|to be published", reference, re.IGNORECASE):
        return "Unpublished"
    if "standard" in ref_lower:
        return "Standard"
    if "dataset" in ref_lower:
        return "Dataset"
    if re.search(r"handbook|manual", reference, re.IGNORECASE):
        return "Handbook"
    if re.search(r"\d+(st|nd|rd|th)\s+ed\.", reference):
        return "Book"
    if "conference" in ref_lower:
        if "presented at" in ref_lower:
            return "Conference Paper (unpublished)"
        return "Conference Proceedings (published)"
    if re.search(r"vol\.\s*\d+", reference) and ("pp." in reference.lower() or "p." in reference.lower()):
        return "Journal"
    return "Other"

# --- Author Extraction ---
def extract_authors(reference):
    match = re.search(r"^\[\d+\]\s*(.*?),\s*“", reference)
    if match:
        authors_str = match.group(1)
        authors_str = authors_str.replace(" and ", ", ")
        authors_list = [author.strip() for author in authors_str.split(",") if author.strip()]
        return authors_list
    return []

# --- Reporting for Journal References ---
def report_journal_details(reference, ref_index):
    authors = extract_authors(reference)
    num_authors = len(authors)
    if num_authors >= 4:
        authors_display = authors[0] + " et al. (Issues: use \"et al.\" when citing in text because there are 4 or more authors)"
    else:
        authors_display = ", ".join(authors)
    
    print(f"Reference [{ref_index}]: {classify_citation(reference)}")
    print("Extracted Details:")
    print("Authors:", authors_display)
    print("Number of Author:", num_authors)
    
    title_match = re.search(r"“(.*?)”", reference)
    if title_match:
        print("Title:", title_match.group(1))
    
    # Extract the journal (or source) name.
    journal_match = re.search(r'“.*?”\s*(.*?)\s*,\s*vol\.', reference)
    if journal_match:
        extracted_journal = journal_match.group(1).strip()
        full_journal, journal_issue = check_full_journal_name(extracted_journal)
        print("Journal/Source:", full_journal, journal_issue)
    
    vol_match = re.search(r"vol\.\s*(\d+)", reference)
    if vol_match:
        print("Volume:", vol_match.group(1))
    
    issue_match = re.search(r"no\.\s*(\d+)", reference)
    if issue_match:
        print("Issue:", issue_match.group(1))
    
    pages_match = re.search(r"p{1,2}\.\s*([\d–-]+)", reference)
    if pages_match:
        pages = pages_match.group(1).strip()
        issue_note_pages = ""
        if "-" in pages or "–" in pages:
            parts = re.split(r"[–-]", pages)
            if len(parts) == 2 and parts[0].strip() == parts[1].strip():
                issue_note_pages = " (Issues: same start and end page)"
        else:
            issue_note_pages = " (Issues: wrong page, no '-' and is single page)"
        print("Page numbers:", pages + issue_note_pages)
    
    year_match = re.search(r'\b(19|20)\d{2}\b', reference)
    if year_match:
        print("Year:", year_match.group(0))
    
    doi_match = re.search(r"doi:\s*(\S+)", reference)
    if doi_match:
        print("DOI:", doi_match.group(1))
    print("------\n")

# --- Reporting for Conference References ---
def report_conference_details(reference, ref_index):
    authors = extract_authors(reference)
    num_authors = len(authors)
    if num_authors >= 4:
        authors_display = authors[0] + " et al. (Issues: use \"et al.\" when citing in text because there are 4 or more authors)"
    else:
        authors_display = ", ".join(authors)
    
    print(f"Reference [{ref_index}]: {classify_citation(reference)}")
    print("Extracted Details:")
    print("Authors:", authors_display)
    print("Number of Author:", num_authors)
    
    title_match = re.search(r"“(.*?)”", reference)
    if title_match:
        print("Title:", title_match.group(1))
    
    # For conference, attempt to extract the conference name from text following "in ".
    conf_match = re.search(r"in\s+([^,]+),", reference, re.IGNORECASE)
    if conf_match:
        print("Conference Name:", conf_match.group(1).strip())
    
    pages_match = re.search(r"p{1,2}\.\s*([\d–-]+)", reference)
    if pages_match:
        pages = pages_match.group(1).strip()
        issue_note_pages = ""
        if "-" in pages or "–" in pages:
            parts = re.split(r"[–-]", pages)
            if len(parts) == 2 and parts[0].strip() == parts[1].strip():
                issue_note_pages = " (Issues: same start and end page)"
        else:
            issue_note_pages = " (Issues: wrong page, no '-' and is single page)"
        print("Page numbers:", pages + issue_note_pages)
    
    year_match = re.search(r'\b(19|20)\d{2}\b', reference)
    if year_match:
        print("Year:", year_match.group(0))
    
    doi_match = re.search(r"doi:\s*(\S+)", reference)
    if doi_match:
        print("DOI:", doi_match.group(1))
    print("------\n")

# --- Reporting for arXiv References ---
def report_arxiv_details(reference, ref_index):
    authors = extract_authors(reference)
    num_authors = len(authors)
    if num_authors >= 4:
        authors_display = authors[0] + " et al. (Issues: use \"et al.\" when citing in text because there are 4 or more authors)"
    else:
        authors_display = ", ".join(authors)
    
    print(f"Reference [{ref_index}]: {classify_citation(reference)}")
    print("Extracted Details:")
    print("Authors:", authors_display)
    print("Number of Author:", num_authors)
    
    title_match = re.search(r"“(.*?)”", reference)
    if title_match:
        print("Title:", title_match.group(1))
    
    year_match = re.search(r'\b(19|20)\d{2}\b', reference)
    if year_match:
        print("Year:", year_match.group(0))
    
    doi_match = re.search(r"doi:\s*(\S+)", reference)
    if doi_match:
        print("DOI:", doi_match.group(1))
    print("------\n")

# --- Generic Reporting for Other Types ---
def report_generic_details(reference, ref_index):
    print(f"Reference [{ref_index}]: {classify_citation(reference)}")
    authors = extract_authors(reference)
    num_authors = len(authors)
    if num_authors >= 4:
        authors_display = authors[0] + " et al. (Issues: use \"et al.\" when citing in text because there are 4 or more authors)"
    else:
        authors_display = ", ".join(authors)
    print("Authors:", authors_display)
    print("Number of Author:", num_authors)
    title_match = re.search(r"“(.*?)”", reference)
    if title_match:
        print("Title:", title_match.group(1))
    print("------\n")

# --- Main Loop: Process Each Reference According to Its Citation Type ---
print("Citation Details Report:\n")
for i, ref in enumerate(references_list, start=1):
    citation_type = classify_citation(ref)
    if citation_type == "Journal":
        report_journal_details(ref, i)
    elif citation_type in ("Conference Proceedings (published)", "Conference Paper (unpublished)"):
        report_conference_details(ref, i)
    elif citation_type == "arXiv":
        report_arxiv_details(ref, i)
    else:
        report_generic_details(ref, i)


Citation Details Report:

Reference [1]: Journal
Extracted Details:
Authors: R. L. Siegel et al. (Issues: use "et al." when citing in text because there are 4 or more authors)
Number of Author: 4
Title: Cancer statistics, 2023,
Journal/Source: CA. Cancer J. Clin. (Issues: Journal name not found in dictionary)
Volume: 73
Issue: 1
Page numbers: 17–48
Year: 2023
DOI: 10.3322/caac.21763.
------

Reference [2]: Journal
Extracted Details:
Authors: K. V. Sriram, R. H. Havaldar
Number of Author: 2
Title: Analytical review and study on object detection techniques in the image,
Journal/Source: Int. J. Model. Simul. Sci. Comput. (Issues: Journal name not found in dictionary)
Volume: 12
Issue: 05
Page numbers: 2150031 (Issues: wrong page, no '-' and is single page)
Year: 2021
DOI: 10.1142/S1793962321500318.
------

Reference [3]: Journal
Extracted Details:
Authors: L. Fan et al. (Issues: use "et al." when citing in text because there are 4 or more authors)
Number of Author: 5
Title: Survey of targ